In [1]:
from selenium import webdriver;
from selenium.webdriver.common.by import By
import os
import pandas as pd
import re
import vk
import time
import pyautogui
from selenium.webdriver.firefox.options import Options
import pyperclip
import json
from dotenv import load_dotenv

In [2]:
df = pd.DataFrame(columns=['text', 'target'])

In [3]:
try:
    load_dotenv()
    
    access_token = os.getenv('VK_ACCESS_TOKEN')
    owner_id = 'stopagent'
    count_posts = 100
    
    # Путь к пользовательскому профилю Chrome
    profile_path = r"C:\Users\Maksim\AppData\Local\Google\Chrome\User Data"
    
    # Настройка опций Chrome
    options = webdriver.ChromeOptions()
    options.add_argument(f"user-data-dir={profile_path}")
    options.add_argument('profile-directory=Default')  # Используем профиль "Default"
    options.add_argument('lang=ru')  # Установка языка интерфейса
    
    # Инициализация WebDriver с использованием настроек
    driver = webdriver.Chrome(options=options)
    
    # Открытие браузера на весь экран
    driver.maximize_window()
    
    driver.get("https://dev.vk.com/ru/method/wall.get")
    
    time.sleep(5)
    
    driver.find_element(By.NAME, "access_token").send_keys(access_token)
    driver.find_element(By.NAME, "owner_id").send_keys(owner_id)

    for i in range(0, 50):
        driver.find_element(By.NAME, "offset").clear()
        driver.find_element(By.NAME, "offset").send_keys(i*count_posts)
        driver.find_element(By.NAME, "count").clear()
        driver.find_element(By.NAME, "count").send_keys(i*count_posts)
        # Нажимаем кнопку "Выполнить"
        driver.find_element(By.CLASS_NAME, "kvyhIw").click()
        
        time.sleep(10)
        from selenium.webdriver.common.action_chains import ActionChains
    
        element = driver.find_element(By.CLASS_NAME, 'sc-caOArR').find_element(By.CLASS_NAME, 'sc-iIaOdA')
        actions = ActionChains(driver)
        actions.move_to_element(element).perform()
        
        # Копируем выведенное
        element.click()
        
        # Получение содержимого буфера обмена
        clipboard_content = pyperclip.paste()
        data = json.loads(clipboard_content)
    
        # Извлечение нужного поля 'text' из каждого элемента в 'items'
        texts = [item.get('text', '') for item in data['response']['items']]
        
        new_df = pd.DataFrame({'text': texts, 'target': '_'})
        df = pd.concat([df, new_df], ignore_index=True)
except Exception as e:
    print(e)

driver.quit()
print(df)

                                                   text target
0     Год английского в школе SkillSet (м. Невский п...      _
1     Сдам квартиру 38 кВ.м.\nЭтаж 4\nЧистое небо,пр...      _
2     Сниму двух комнатную квартиру! Ориентир 7-8 ию...      _
3     Сдаётся однокомнатная квартира. На длительный ...      _
4     Семья 3 человека, Снимем однокомнатную или дву...      _
...                                                 ...    ...
4915  Сдам 1 к. к. квартиру, желательно славянам. Кв...      _
4916  Сдается однокомнатная квартира по адресу : Пол...      _
4917  http://stopagent.ru/id1046956\n\nм.Комендантск...      _
4918  http://stopagent.ru/id1046848\n\nм.Кировский з...      _
4919  Сдается на долгий срок (без комиссии, без аген...      _

[4920 rows x 2 columns]


In [11]:
df = df.drop_duplicates()
pd.set_option('display.max_colwidth', None)
df

,text,target
1,"Сдам квартиру 38 кВ.м.\nЭтаж 4\nЧистое небо,проспект Авиаконструкторов,69\n28000+к/у\nРемонт от застройщика,без мебели\n89191437589",Сдать
2,"Сниму двух комнатную квартиру! Ориентир 7-8 июля \nВ шаговой доступности от метро \nО себе: Парень 24 года, финансово обеспеченный, опрятный не курящий и не пьющий, не студент.\nБардель и прочие тусовки не устраиваю\nПроживать буду с девушкой без животных и детей \nЦена в пределах разумного 40-50 тысяч рублей+коммунальные",Снять
3,"Сдаётся однокомнатная квартира. На длительный срок ! По дрговору. Метро рыбацкое, Советский проспект ( Усть - Славянка ) дом 41, кор. 1,10 этаж. 19 тыс. + к.у. Все необходимое есть . Залог 19 тысяч. Агентов просьба не беспокоить.",Сдать
4,"Семья 3 человека, Снимем однокомнатную или двухкомнатную квартиру на длительный период. на прямую от собственника. Цена до 40 т.р.\nТериториально станции метро: Политехническая, Академическая, гр Проспект, в пешей доступности от метро. \nРассмотрю любые варианты. \n8 920 298 18 42 Рафаэль",Снять
5,"Сниму КОМНАТУ в 2х комнатной или 3х комнатной квартире, сумма до 20 000 с комами, желательно не далеко от метро. Заехать готова после +- 13 июля. Не курю, не пью, шумные вечеринки не устраиваю. Почти постоянно на работе, поэтому дискомфорта не доставлю.",Снять
...,...,...
4913,"http://stopagent.ru/id1047224\n\nм.Комендантск.пр. , проспект Испытателей, 26\n\nСдам квартиру семье. 20000 Плюс квартплата. Квартира светлая, тёплая, чистая. Дверь двойная. Плюс железная отделяет этаж от лифта. Мебели минимум: диван двухспальный удобный, шифоньер, стеллаж, тумба, тв, рабочий столик. В кухне: холодильник, электрон плита, шкафы, стол, табуреты. Есть минимум посуды. Если мебель не нужна -можно убрать. Стиральная машина Bosch. Пылесос. Балкон открытый длинный вдоль всей квартиры. Зелёный ухоженный двор. На длительный срок. Без посредников. Беру залог. Возможна какая-то небольшая косметика в счёт квартплаты.\n\n79217840048",Сдать
4915,"Сдам 1 к. к. квартиру, желательно славянам. Квартира 42 метра, просторная кухня. Мебели и бытовой техники нет, но по договорённости можно решить этот вопрос.\nКвартира находится в деревне Горбунки, Ленинградской области. От метро Проспект Ветеранов на маршрутном автобусе 30 мин. Рядом Петродворец, фонтаны. В деревне Горбунки есть все что нужно для жизни, очень компактно насыщенный социальными объектами посёлок (школа, детский сад, университет, лицей, музыкальная школа, дворец культуры, два современных стадиона, хоккейная коробка, спортивный комплекс, библиотека, поликлиника, парк ""Беззаботное"", где летом откроется лодочная станция.)",Сдать
4916,"Сдается однокомнатная квартира по адресу : Полевая Сабировская, д.47. 22 этаж. Я собственник! Видовая. 33 кв.м. балкон застеклен. техника. мебель. 23 000+ ку. До метро Комендантский проспект 10 мин пешком. Также недалёко метро пионерская и старая деревня. Напротив дома окей, Максидом.",Сдать
4917,"http://stopagent.ru/id1046956\n\nм.Комендантск.пр. , Комендантский проспект, 71\n\nОт собственника, без комиссии. Сдам на длительный срок студию 24 м2 в новом жк ""Чистое небо"" (Приморский район, Комендантский пр. 71). В квартире есть всё необходимое для проживания: новая мебель и бытовая техника (диван, шкаф, кухня с обеденной зоной, варочная панель, холодильник, стиральная машина, оборудованный санузел, душевая кабина). Большая застекленная лоджия 6 м2. Рядом «Пятерочка», «Лента», К-раута». Маршрутка до метро Комендантский проспект (179К) – 10 минут, Пионерская (79К)– 20-25 минут, кольцо маршруток рядом с домом. Удобный выезд на кад и зсд. Оплата 18 тыс. руб. + ку (установлены счетчики на воду, тепло- и электроэнергию)+залог 18 т. р. Сдам гражданам рф. Показы по договоренности после 20-00 в будние дни, в выходные в любое время.\n\n79110278351",Сдать


In [6]:
df.loc[df['text'].str.lower().str.contains('сниму'), 'target'] = 'Снять'
df.loc[df['text'].str.lower().str.contains('снимем'), 'target'] = 'Снять'
df.loc[df['text'].str.lower().str.contains('ищем'), 'target'] = 'Снять'
df.loc[df['text'].str.lower().str.contains('снимет'), 'target'] = 'Снять'
df.loc[df['text'].str.lower().str.contains('ищу'), 'target'] = 'Снять'
df.loc[df['text'].str.lower().str.contains('поиск'), 'target'] = 'Снять'
df.loc[df['text'].str.lower().str.contains('ищет'), 'target'] = 'Снять'
df.loc[df['text'].str.lower().str.contains('нужна'), 'target'] = 'Снять'

df.loc[df['text'].str.lower().str.contains('сдам'), 'target'] = 'Сдать'
df.loc[df['text'].str.lower().str.contains('сдаётся'), 'target'] = 'Сдать'
df.loc[df['text'].str.lower().str.contains('сдается'), 'target'] = 'Сдать'
df.loc[df['text'].str.lower().str.contains('сдаю'), 'target'] = 'Сдать'
df.loc[df['text'].str.lower().str.contains('сдаем'), 'target'] = 'Сдать'

C:\Users\Maksim\AppData\Local\Temp\ipykernel_11940\3144188238.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[df['text'].str.lower().str.contains('сниму'), 'target'] = 'Снять'
C:\Users\Maksim\AppData\Local\Temp\ipykernel_11940\3144188238.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[df['text'].str.lower().str.contains('снимем'), 'target'] = 'Снять'
C:\Users\Maksim\AppData\Local\Temp\ipykernel_11940\3144188238.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_gui

In [12]:
df = df[df['target'] != '_']

In [14]:
df.describe()

,text,target
count,1766,1766
unique,1766,2
top,"Сдам квартиру 38 кВ.м.\nЭтаж 4\nЧистое небо,проспект Авиаконструкторов,69\n28000+к/у\nРемонт от застройщика,без мебели\n89191437589",Сдать
freq,1,1390


In [37]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Разделение данных на признаки (X) и метки (y)
X = df['text']
y = df['target']

# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Преобразование текстовых данных в числовые векторы
vectorizer = CountVectorizer()
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

# Создание и обучение наивного байесовского классификатора
model = MultinomialNB()
model.fit(X_train_vectorized, y_train)

# Предсказание на тестовых данных
y_pred = model.predict(X_test_vectorized)

# Оценка точности модели
accuracy = accuracy_score(y_test, y_pred)
print(f'Точность модели: {accuracy:.2f}')

Точность модели: 1.00
Объявление: "куплю старый телефон" классифицировано как: Снять


In [52]:
# Пример классификации нового объявления
new_ad = "Добрый день! Сдам"
new_ad_vectorized = vectorizer.transform([new_ad])
predicted_probabilities = model.predict_proba(new_ad_vectorized)

# Получение списка классов
classes = model.classes_

# Вывод вероятностей для каждого класса
for cls, prob in zip(classes, predicted_probabilities[0]):
    print(f'Класс: {cls}, Вероятность: {prob:.2f}')

# Вывод предсказанного класса
predicted_label = model.predict(new_ad_vectorized)
print(f'Объявление: "{new_ad}" классифицировано как: {predicted_label[0]}')

Класс: Сдать, Вероятность: 0.07
Класс: Снять, Вероятность: 0.93
Объявление: "Добрый день! Сдам" классифицировано как: Снять
